In [6]:
import json
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, accuracy_score)

In [59]:
df = pd.read_csv('data/weather_data_hcm_[2017-2022]_preprocess.csv')

In [60]:
df.head()

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,00:00,1,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,Partly cloudy
1,03:00,1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,Partly cloudy
2,06:00,1,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,Partly cloudy
3,09:00,1,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,Partly cloudy
4,12:00,1,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,Partly cloudy


In [9]:
label = df['Weather']
features = df

In [10]:
#remove data
features = features[:-1]
label = label[1:]

In [11]:
data = features
data['Label'] = label.values

/tmp/ipykernel_2296341/1655283038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = label.values


In [12]:
RAINY_LABEL = ['Light drizzle','Light rain','Light rain shower', 'Patchy light drizzle', 'Patchy light rain',
             'Patchy light rain with thunder','Patchy rain possible','Heavy rain','Heavy rain at times',
             'Moderate or heavy rain shower','Moderate rain', 'Moderate rain at times', 'Overcast','Torrential rain shower']
RAINY_MONTH = [5,6,7,8,9,10,11]
MORNING_HOUR = [6,9,12,15]

In [13]:
for i, value in enumerate(data.Label):
    if value in RAINY_LABEL:
        data.Label[i] = 'yes'
    else:
        data.Label[i] = 'no'

/tmp/ipykernel_2296341/4224172725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[i] = 'no'
/data4/thang/envs/py38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2296341/4224172725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[i] = 'yes'


In [14]:
# for i, value in enumerate(data.Time):
#     hour = value.split(':')[0]
#     data.Time[i] = hour

In [15]:
for i, value in enumerate(data.Weather):
    if value in RAINY_LABEL:
        data.Weather[i] = 'rainy'
    else:
        data.Weather[i] = 'sunny'

/tmp/ipykernel_2296341/812369229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weather[i] = 'sunny'
/tmp/ipykernel_2296341/812369229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Weather[i] = 'rainy'


In [16]:
# for i, value in enumerate(data.Month):
#     if int(value) in RAINY_MONTH:
#         data.Month[i] = 'rainy_month'
#     else:
#         data.Month[i] = 'sunny_month'

In [17]:
# remove columns
# drop_columns = ['Day', 'Year', 'Humidity', 'Pressure']
drop_columns = []
data = data.drop(drop_columns, axis=1)
data.head()

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather,Label
0,00:00,1,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,sunny,no
1,03:00,1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,sunny,no
2,06:00,1,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,sunny,no
3,09:00,1,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,sunny,no
4,12:00,1,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,sunny,no


In [18]:
def convert_category2numeric(data, columns_name):
    for col in columns_name:
        data = pd.concat((data, pd.get_dummies(data[col], prefix=col)), axis=1)
        data = data.drop([col], axis=1)
    return data

In [19]:
data = convert_category2numeric(data, ['Time', 'Weather', 'Month'])
data = pd.concat((data, pd.get_dummies(data['Label'], prefix='Label', drop_first=True)), axis=1)
data = data.drop(['Label'], axis=1)
data.head()

,Day,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,...,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Label_yes
0,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,...,0,0,0,0,0,0,0,0,0,0
1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,...,0,0,0,0,0,0,0,0,0,0
2,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,...,0,0,0,0,0,0,0,0,0,0
3,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,...,0,0,0,0,0,0,0,0,0,0
4,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
all_columns = data.columns.to_list()
Y_colums = ['Label_yes']
X_colums = [x for x in all_columns if x not in Y_colums]

In [21]:
train_set, test_set = train_test_split(data, test_size=0.2)

In [22]:
Xtrain = train_set[X_colums]
ytrain = train_set[Y_colums]

# building the model and fitting the data
log_reg = sm.Logit(ytrain, Xtrain).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: inf
         Iterations 8


/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1872: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/data4/thang/envs/py38/lib/python3.8/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Label_yes   No. Observations:                11408
Model:                          Logit   Df Residuals:                    11377
Method:                           MLE   Df Model:                           30
Date:                Mon, 25 Jul 2022   Pseudo R-squ.:                     inf
Time:                        13:41:40   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
Covariance Type:            nonrobust   LLR p-value:                     1.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Day               0.0012      0.003      0.371      0.710      -0.005       0.007
Year             -0.0928      0.019     -4.994      0.000      -0.129      -0.056
Humidity          0.0067      0.004      1.605      0.108      -0.001       0.015
Dew point         0.1306      0.025      5.172      0.000       0.081       0.180
Temperature       0.0134      0.020      0.668      0.504      -0.026       0.053
Rain              0.3725      0.030     12.599      0.000       0.315       0.430
Cloud             0.0382      0.001     27.165      0.000       0.035       0.041
Pressure         -0.0448      0.019     -2.324      0.020      -0.083      -0.007
Wind             -0.0551      0.027     -2.063      0.039      -0.107      -0.003
Gust              0.0610      0.019      3.186      0.001       0.023       0.099
Dir               0.0005      0.000      1.336      0.182      -0.000       0.001
Time_00:00       39.3747        nan        nan        nan         nan         nan
Time_03:00       39.5309        nan        nan        nan         nan         nan
Time_06:00       39.4229        nan        nan        nan         nan         nan
Time_09:00       39.9372        nan        nan        nan         nan         nan
Time_12:00       40.5887        nan        nan        nan         nan         nan
Time_15:00       40.5995        nan        nan        nan         nan         nan
Time_18:00       39.4426        nan        nan        nan         nan         nan
Time_21:00       39.7340        nan        nan        nan         nan         nan
Weather_rainy   159.7319        nan        nan        nan         nan         nan
Weather_sunny   158.8988        nan        nan        nan         nan         nan
Month_1          25.8602        nan        nan        nan         nan         nan
Month_2          26.1641        nan        nan        nan         nan         nan
Month_3          25.9216        nan        nan        nan         nan         nan
Month_4          26.5125        nan        nan        nan         nan         nan
Month_5          26.6230        nan        nan        nan         nan         nan
Month_6          26.8630        nan        nan        nan         nan         nan
Month_7          26.5118        nan        nan        nan         nan         nan
Month_8          26.5751        nan        nan        nan         nan         nan
Month_9          27.0523        nan        nan        nan         nan         nan
Month_10         27.2317        nan        nan        nan         nan         nan
Month_11         26.8585        nan        nan        nan         nan         nan
Month_12         26.4569        nan        nan        nan         nan         nan
=================================================================================
"""

In [23]:
Xtest = test_set[all_columns[:-1]]
ytest = test_set[all_columns[-1]]

yhat = log_reg.predict(Xtest)
prediction = list(map(round, yhat))
cm = confusion_matrix(ytest, prediction) 
print ("Confusion Matrix : \n", cm)

# accuracy score of the model
print('Test accuracy = ', accuracy_score(ytest, prediction))

Confusion Matrix : 
 [[1615  232]
 [ 304  701]]
Test accuracy =  0.8120617110799438


In [24]:
log_reg.save('log_model.pickle')

In [25]:
log_model = sm.load('log_model.pickle')

In [26]:
a = dict(Xtest.iloc[[0]])

In [27]:
inputs = {}
for k,v in a.items():
    inputs[k] = v.values[0]

In [28]:
x = pd.DataFrame([inputs])

In [57]:
x.columns

Index(['Day', 'Year', 'Humidity', 'Dew point', 'Temperature', 'Rain', 'Cloud',
       'Pressure', 'Wind', 'Gust', 'Dir', 'Time_00:00', 'Time_03:00',
       'Time_06:00', 'Time_09:00', 'Time_12:00', 'Time_15:00', 'Time_18:00',
       'Time_21:00', 'Weather_rainy', 'Weather_sunny', 'Month_1', 'Month_2',
       'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8',
       'Month_9', 'Month_10', 'Month_11', 'Month_12'],
      dtype='object')

In [30]:

y = log_model.predict(x)
y = list(map(round, y))

,Time,Day,Month,Year,Humidity,Dew point,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,00:00,1,1,2020,65.0,19.0,27.0,0.0,10.0,1015.0,8.0,11.0,46.6,sunny
1,03:00,1,1,2020,69.0,19.0,25.0,0.0,19.0,1014.0,8.0,13.0,26.9,sunny
2,06:00,1,1,2020,78.0,18.0,24.0,0.0,10.0,1015.0,8.0,11.0,29.2,sunny
3,09:00,1,1,2020,55.0,19.0,29.0,0.0,5.0,1016.0,7.0,8.0,66.7,sunny
4,12:00,1,1,2020,30.0,14.0,34.0,0.0,8.0,1015.0,10.0,11.0,73.0,sunny


In [55]:
with open('test_sample.json', 'w') as f:
    json.dump(test_sample, f)

In [ ]:
df = pd.read_excel('data/weather_data_VietNam_[2021].xlsx')

In [ ]:
df.head()

,Time,Humidity,Dew point,Day,Month,year,Province,Temperature,Rain,Cloud,Pressure,Wind,Gust,Dir,Weather
0,00:00:00,65,19,1,1,2021,TÃ¢y Ninh,23,0.0,26,1012,18,29,12.8,Partly cloudy
1,03:00:00,69,19,1,1,2021,TÃ¢y Ninh,22,0.0,32,1011,17,26,11.4,Partly cloudy
2,06:00:00,78,18,1,1,2021,TÃ¢y Ninh,21,0.0,43,1012,15,25,8.0,Partly cloudy
3,09:00:00,55,19,1,1,2021,TÃ¢y Ninh,24,0.0,36,1013,19,24,15.0,Partly cloudy
4,12:00:00,30,14,1,1,2021,TÃ¢y Ninh,27,0.0,28,1012,19,22,14.1,Partly cloudy
